<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np

# Количество примеров
n = 1000
np.random.seed(42)

# Генерация числовых признаков
estate_worth = np.random.uniform(100000, 1000000, n).round(2)
debts = np.random.uniform(0, 200000, n).round(2)
funeral_expenses = np.random.uniform(5000, 10000, n).round(2)
bequests = np.random.uniform(1000, 200000, n).round(2)

# Создание упрощённого текстового описания
texts = [
    f"Estate: ${ew:.2f}, Debts: ${d:.2f}, Funeral: ${fe:.2f}, Bequest: ${b:.2f}."
    for ew, d, fe, b in zip(estate_worth, debts, funeral_expenses, bequests)
]

# Формирование DataFrame
df_synthetic = pd.DataFrame({
    'text': texts,
    'estate_worth': estate_worth,
    'debts': debts,
    'funeral_expenses': funeral_expenses,
    'bequests': bequests
})

# Создание бинарной метки по медиане bequests
median_bequests = df_synthetic['bequests'].median()
df_synthetic['label'] = (df_synthetic['bequests'] >= median_bequests).astype(int)

# Сохраняем датасет в CSV-файл
output_path = 'synthetic_inheritance_dataset.csv'
df_synthetic.to_csv(output_path, index=False)

print(f"Синтетический датасет сохранён в файл: {output_path}")
print(df_synthetic.head())


Синтетический датасет сохранён в файл: synthetic_inheritance_dataset.csv
                                                text  estate_worth      debts  \
0  Estate: $437086.11, Debts: $37026.59, Funeral:...     437086.11   37026.59   
1  Estate: $955642.88, Debts: $108380.19, Funeral...     955642.88  108380.19   
2  Estate: $758794.55, Debts: $174589.17, Funeral...     758794.55  174589.17   
3  Estate: $638792.64, Debts: $146444.98, Funeral...     638792.64  146444.98   
4  Estate: $240416.78, Debts: $161312.23, Funeral...     240416.78  161312.23   

   funeral_expenses   bequests  label  
0           6308.53  134867.90      1  
1           6234.89  159539.60      1  
2           9531.27   50843.11      0  
3           6247.73  125349.95      1  
4           6359.75  114777.45      1  


In [20]:
import pandas as pd

# Предположим, df - ваш DataFrame с очищенными данными
# Например, df = df_synthetic из предыдущих шагов

# Сохраняем датасет в CSV-файл
csv_filename = 'clean_synthetic_dataset.csv'
df.to_csv(csv_filename, index=False)
print(f"Датасет сохранён в файл {csv_filename}")

# Для скачивания файла в Jupyter Notebook (Google Colab)
try:
    from google.colab import files
    files.download(csv_filename)
except ImportError:
    print("Автоматическая загрузка доступна только в Google Colab. Файл сохранён локально.")


Датасет сохранён в файл clean_synthetic_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>